In [1]:
import requests
from bs4 import BeautifulSoup
import string
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from newspaper import Article
import newspaper
import json
pd.set_option('max_colwidth', 100)

In [2]:
#BeautifulSoup part

URL = 'https://rusdisinfo.voxukraine.org'
content = requests.get(URL).text
soup = BeautifulSoup(content, "html.parser")
publications = soup.find(class_='Home_container__bCOhY')
keys = publications.find_all('h2', class_ = "FakeItem_fakeHeading__Uc0QN")
values = publications.find_all('a')
valid_links = []
for v in values:
    link = URL + v.get('href')
    r = requests.get(link)
    if r.status_code != 404:
        valid_links.append(link)

labeled_dict = {}

for index, k in enumerate(keys):
    k = keys[index].text[2:]
    v = valid_links[index]      
    labeled_dict.update({k:v})

In [3]:
# Selenium part

DRIVER_PATH = '/path/to/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://google.com')

final_dictionary = {}

for k,v in labeled_dict.items():
    driver.get(v)
    all_links = driver.find_elements(By.CLASS_NAME, 'Narrative_fakeLink___YbTe')
    for i in all_links:
        i.click()
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_dirty = soup.find_all(class_ = 'Narrative_media__B2gNZ')
    final_list = []
    for u in list_dirty:
        link_news = u.find('a').get('href')
        final_list.append(link_news)

    final_dictionary.update({k:final_list})

/var/folders/4b/j_t85q4j2klgsp_lqvw39xk40000gn/T/ipykernel_76149/3067093557.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [4]:
# Newspaper3k - python library for extracting & curating articles

list_scraped = []

for label, links in final_dictionary.items():
    
    # Current limit to 3 news from each category
    
    for url in links[:3]:
        article = newspaper.Article(url=url)
        article.download()

        try:
            article.parse()
            article ={"title": str(article.title), "text": str(article.text)}
            article.update({"link":url})
            article.update({"label": label})
            list_scraped.append(article)

        except:
            continue

In [9]:
df = pd.DataFrame(list_scraped)
df.head(10)

,title,text,link,label
0,Krieg im Donbass: Ukraine verkündet das Ende des Abkommens von Minsk – Anti-Spiegel,"Präsident Selensky hat de facto das Ende des Abkommens von Minsk verkündet mitteilen lassen, sei...",https://www.anti-spiegel.ru/2021/krieg-im-donbass-ukraine-verkuendet-das-ende-des-abkommens-von-...,Maidan in 2014 was a coup d'état
1,Die Lebkuchenhexe aus Washington – Jetzt intrigiert sie gegen Nord Stream 2,Die Lebkuchenhexe aus Washington – Jetzt intrigiert sie gegen Nord Stream 2\n\nDa ist sie wieder...,https://de.rt.com/meinung/129862-lebkuchenhexe-im-weissen-haus/,Maidan in 2014 was a coup d'état
2,Die NATO rückt in der Ukraine immer näher an einen Krieg gegen Russland,"Was die Ukraine anbelangt, so scheint Ankara das Tempo für die verstärkte Einmischung der NATO i...",https://uncutnews.ch/die-nato-rueckt-in-der-ukraine-immer-naeher-an-einen-krieg-gegen-russland/,Nazis and ultra-nationalists in Ukraine
3,Bielorussia nel mirino dei barbari di Washington: cosa succede?,Ciò che sta accadendo in questi giorni in Bielorussia è la fotocopia esatta di quanto è già acca...,https://www.radioradio.it/2020/08/bieloRussia-nel-mirino-dei-barbari-di-washington-cosa-succede/,Nazis and ultra-nationalists in Ukraine
4,Die US-Kriegspläne für die Ukraine,"Offen gesagt, Bidens Ansprache an die US-Nation (die erste in seinem Leben) war wahrscheinlich d...",https://uncutnews.ch/die-us-kriegsplaene-fuer-die-ukraine/,Nazis and ultra-nationalists in Ukraine
5,Bloomberg – La Germania non ha promesso agli Stati Uniti di sospendere Nord Stream 2 a causa del...,La Germania ha negato di aver promesso agli Stati Uniti di bloccare il gasdotto ‘Nord Stream 2’ ...,https://resetitaliablog.altervista.org/bloomberg-la-germania-non-ha-promesso-agli-stati-uniti-di...,Nord Stream-2
6,Il prezzo del gas resta a livelli record,"Il prezzo del gas resta a livelli record\n\nTra le ragioni dell’impennata: il freddo, la crisi u...",https://www.corrierenazionale.net/il-prezzo-del-gas-resta-a-livelli-record/,Nord Stream-2
7,Il prezzo del gas resta a livelli record,"Il prezzo del gas resta a livelli record\n\nTra le ragioni dell’impennata: il freddo, la crisi u...",https://www.corrierenazionale.net/il-prezzo-del-gas-resta-a-livelli-record/,Nord Stream-2
8,"Mehr Fluggeräte, Schiffe und Militärübungen - Ukraine stockt Nato-Präsenz 2022 auf","Mehr Fluggeräte, Schiffe und Militärübungen - Ukraine stockt Nato-Präsenz 2022 auf\n\nDie Ukrain...",https://de.rt.com/europa/128476-ukraine-wird-im-jahr-2022-nato-pr%C3%A4senz-land-erheblich-erwei...,Russia needs to protect itself
9,Ukraine als Aufmarschgebiet der NATO: Bereits 10.000 Soldaten der westlichen Allianz im Land,Ukraine als Aufmarschgebiet der NATO: Bereits 10.000 Soldaten der westlichen Allianz im Land\n\n...,https://de.rt.com/europa/128523-ukraine-als-aufmarschplatz-nato-schon-10000-soldaten-der-westlic...,Russia needs to protect itself
